In [2]:
import sys
sys.path.append('../libs')
from src import dhis_meta as mt
import utils as fn


conf=fn.get_config('/home/n2/.dih.configs/jna-dhis-config.json')
meta=mt.DHIS_Meta(conf);
# meta.add_category_combo()
# meta.rename_elements()
meta.update()
# meta.__map

.

In [ ]:
import sys
sys.path.append('../libs')
from src import dhis_meta as mt
import utils as fn

